In [2]:
import pandas as pd
import numpy as np
import os
import sys
import psutil
from time import time
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import resource
from glob import glob
from IPython.display import clear_output

from gpflow.models.gpr import GPR

## Data preperation

In [3]:
main_path = '~/Nonstat-exps/gp_extra/'
df = pd.read_csv(main_path+'data/beijing_AQI.csv').rename(columns={'PM25_Concentration':'PM25','longitude':'long','latitude':'lat'})
df = df.set_index('time').sort_index()
print('unique timestamps are',len(df.index.unique()))
useful_ts = []
for ts in df.index.unique():
  if(len(df.loc[ts])==36):
    useful_ts.append(ts)
df = df.loc[useful_ts]
df['PM25'] = df['PM25'].astype(float)
print('unique timestamps after removing missing entry time-stamps are',len(useful_ts))
df.columns

n_ts = len(useful_ts) # Number of timestamps
K = 4 #  Number of folds
n_val = 2 # Number of validation stations

splitter = KFold(K, shuffle=True, random_state=0)
stations = np.sort(df['station_id'].unique())
folds={i:{'train':None,'val':None,'test':None} for i in range(K)}
for i, (train_val, test) in enumerate(splitter.split(stations)):
    folds[i]['train'] = stations[train_val[:-n_val]]
    folds[i]['val'] = stations[train_val[-n_val:]]
    folds[i]['test'] = stations[test]
    
###########################
# Data preperation
###########################
data = {i:{'train_Xy':None,'val_Xy':None,'test_Xy':None} for i in range(K)}
for fold in range(K):
    for part in ['train','val','test']:
        data[fold][part+'_Xy'] = (df[df.station_id.isin(folds[fold][part])][['long', 'lat']], 
                                  df[df.station_id.isin(folds[fold][part])][['PM25']])

unique timestamps are 7460
unique timestamps after removing missing entry time-stamps are 2132


## K fold : Kriging

In [5]:
path = 'results/raw_kriging/'
print(len(glob(path+'/*')))

8512


### Training

In [ ]:
jobs = []
for fold in range(K):
    for ts in range(n_ts):
        if not os.path.exists(path+'ts_'+str(df.index.unique()[ts])+'_fold_'+str(fold)):
            jobs.append('python scripts/process_kriging.py {0} {1}'.format(ts, fold))

In [15]:
print('starting',len(jobs),'jobs on',psutil.cpu_count(),'CPUs')
init = time()
maxa = 0
while len(glob(path+'/*')) < len(useful_ts):
    if maxa>10:
        break
    for i,j in zip(np.arange(0,len(jobs)+1,5), np.arange(0,len(jobs)+1,5)[1:]):
        print('from',i,'to',j)
        os.system(' | '.join(jobs[i:j]))
    print('round complete')
    maxa+=1
print((time()-init)/60, 'all fold complete')

starting 15 jobs on 32 CPUs
0.0004952232042948405 all fold complete


In [10]:
jobs

['python scripts/process_kriging.py 2116 3',
 'python scripts/process_kriging.py 2117 3',
 'python scripts/process_kriging.py 2118 3',
 'python scripts/process_kriging.py 2119 3',
 'python scripts/process_kriging.py 2120 3',
 'python scripts/process_kriging.py 2121 3',
 'python scripts/process_kriging.py 2122 3',
 'python scripts/process_kriging.py 2123 3',
 'python scripts/process_kriging.py 2124 3',
 'python scripts/process_kriging.py 2125 3',
 'python scripts/process_kriging.py 2126 3',
 'python scripts/process_kriging.py 2127 3',
 'python scripts/process_kriging.py 2128 3',
 'python scripts/process_kriging.py 2129 3',
 'python scripts/process_kriging.py 2130 3',
 'python scripts/process_kriging.py 2131 3']

### RMSE calculation

In [16]:
preds = []
tests = []
for fold in range(K):
    tmp_preds = []
    tmp_tests = []
    for ts_n, ts in enumerate(df.index.unique()[:n_ts]):
        tmp = pd.read_pickle(path+'ts_'+str(ts)+'_fold_'+str(fold))
        preds.append(tmp['pred_y'].squeeze())
        tests.append(tmp['test_y'].squeeze())
        tmp_preds.append(tmp['pred_y'].squeeze())
        tmp_tests.append(tmp['test_y'].squeeze())
    print("Fold",fold,'rmse',mean_squared_error(np.array(tmp_tests).flatten(), np.array(tmp_preds).flatten(), squared=False))
print("Overall RMSE", mean_squared_error(np.array(tests).flatten(), np.array(preds).flatten(), squared=False))

Fold 0 rmse 27.165096537284
Fold 1 rmse 38.41403483373693
Fold 2 rmse 25.639915050686692
Fold 3 rmse 32.46254746570393
Overall RMSE 31.324123187477348


## K fold GP-RBF

In [6]:
path = 'results/raw_gp_rbf/'

### Training

In [7]:
jobs = []
for fold in range(K):
    for ts in range(n_ts):
        jobs.append('python scripts/process_gp_rbf.py {0} {1}'.format(ts, fold))

print('starting',len(jobs),'jobs on',psutil.cpu_count(),'CPUs')
init = time()
maxa = 0
while len(glob(path+'/*')) != len(jobs)*2:
    if maxa>10:
        break
    os.system(' | '.join(jobs))
    print('round complete')
    maxa+=1
print((time()-init)/60, 'all fold complete')

starting 96 jobs on 32 CPUs
2.770821253458659e-05 all fold complete


### RMSE calculation

In [8]:
preds = []
tests = []
hyp = []
for fold in folds:
    tmp_preds = []
    tmp_tests = []
    for ts_n, ts in enumerate(df.index.unique()[:n_ts]):
        tmp = pd.read_pickle(path+'ts_'+str(ts)+'_fold_'+str(fold))
        hyp.append(tmp['best_hyperpara']['ls_init'])
        preds.append(tmp['pred_y'].squeeze())
        tests.append(tmp['test_y'].squeeze())
        tmp_preds.append(tmp['pred_y'].squeeze())
        tmp_tests.append(tmp['test_y'].squeeze())
    print("Fold",fold,'rmse',mean_squared_error(np.array(tmp_tests).flatten(), np.array(tmp_preds).flatten(), squared=False))
print("Overall RMSE", mean_squared_error(np.array(tests).flatten(), np.array(preds).flatten(), squared=False))
pd.Series(hyp).value_counts()

Fold 0 rmse 26.56725659802779
Fold 1 rmse 18.633502800778835
Fold 2 rmse 22.446714127157588
Fold 3 rmse 25.487030306341303
Overall RMSE 23.48653996821786


100.00    37
1.00      24
0.10      19
10.00     14
0.01       2
dtype: int64

## K fold GP-LLS

In [9]:
path = 'results/raw_gp_lls/'

### Training

In [ ]:
jobs = []
for fold in range(K):
    for ts in range(n_ts):
        jobs.append('python scripts/process_gp_lls.py {0} {1}'.format(ts, fold))

print('starting',len(jobs),'jobs on',psutil.cpu_count(),'CPUs')
init = time()
maxa = 0
while len(glob(path+'/*')) != len(jobs)*2:
    if maxa>10:
        break
#     os.system(' | '.join(jobs))
    for b_id, batch in enumerate(np.array_split(jobs, 24)):
        print("batch of",len(batch),'started')
        os.system(' | '.join(batch))
        clear_output(wait=True)
        print(b_id)
        print((time()-init)/60, 'minutes: a batch complete')
#     for j_id,job in enumerate(jobs):
#         os.system(job)
#         clear_output(wait=True)
#         print(j_id)
    print('round complete')
    maxa+=1
print((time()-init)/60, 'all fold complete')

16
45.39513452450434 minutes: a batch complete
batch of 4 started


### RMSE calculation

In [ ]:
preds = []
tests = []
hyp = []
for fold in folds:
    tmp_preds = []
    tmp_tests = []
    for ts_n, ts in enumerate(df.index.unique()[:n_ts]):
        tmp = pd.read_pickle(path+'ts_'+str(ts)+'_fold_'+str(fold))
        hyp.append(tmp['best_hyperpara']['N'])
        preds.append(tmp['pred_y'].squeeze())
        tests.append(tmp['test_y'].squeeze())
        tmp_preds.append(tmp['pred_y'].squeeze())
        tmp_tests.append(tmp['test_y'].squeeze())
    print("Fold",fold,'rmse',mean_squared_error(np.array(tmp_tests).flatten(), np.array(tmp_preds).flatten(), squared=False))
print("Overall RMSE", mean_squared_error(np.array(tests).flatten(), np.array(preds).flatten(), squared=False))
pd.Series(hyp).value_counts()

# Appendix

In [ ]:
# for file in glob(path+'/*'):
#     try:
#         print(mean_squared_error(pd.read_pickle(file)['test_y'], pd.read_pickle(file)['pred_y'], squared=False))
#     except:
#         pass

In [ ]:
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# N=3
# model = GPR(scaler.fit_transform(data[fold]['train_Xy'][0].loc[ts].values), 
#                     data[fold]['train_Xy'][1].loc[ts].values,
#                    LLS(2, scaler.fit_transform(data[fold]['train_Xy'][0].loc[ts].values), N, active_dims=[0,1]))